# Current Quarter

<b>01 - Increase total amount of service on the SHN and reliability of that service by 2024</b>

## Routes on the State Highway Network (SHN)

Transit routes along the SHN can be categorized into 3 groups:
1. **On SHN** - where at least 20% of the transit route runs the SHN (within 50 ft) 
2. **Intersects SHN** - where at least 35% of the transit route runs within 0.5 mile of the SHN.
3. **Other** - all other transit routes.

### Metrics
* service hours, service hours per route
* delay hours, delay hours per route
* map of route by category (and by mode)

The metrics are shown for for transit routes **on the SHN** and **intersects SHN**.

In [ ]:
%%capture
import warnings
warnings.filterwarnings('ignore')

import calitp.magics
import geopandas as gpd
import intake
import pandas as pd

from IPython.display import HTML

import B1_report_metrics as report_metrics
import B2_report_charts as report_charts
from update_vars import BUS_SERVICE_GCS
from shared_utils import geography_utils, portfolio_utils, rt_dates
from shared_utils import calitp_color_palette as cp
from bus_service_utils import chart_utils

hq_catalog = intake.open_catalog("../high_quality_transit_areas/*.yml")

In [ ]:
# parameters cell
CURRENT_QUARTER = "Q3_2022"
ANALYSIS_DATE = rt_dates.PMAC[CURRENT_QUARTER]

In [ ]:
%%capture_parameters
QUARTER_CLEANED = CURRENT_QUARTER.replace('_', ' ')
CURRENT_QUARTER, ANALYSIS_DATE, QUARTER_CLEANED

In [ ]:
df = report_metrics.prep_data_for_report(ANALYSIS_DATE)

## Statewide Stats for {QUARTER_CLEANED} ({ANALYSIS_DATE})

In [ ]:
summary = report_metrics.get_service_hours_summary_table(df)  

In [ ]:
all_hours = geography_utils.aggregate_by_geography(
    summary.assign(category="All"),
    group_cols = ["category"],
    sum_cols = ["unique_route", "service_hours"]
)

In [ ]:
STATEWIDE_HOURS = all_hours.service_hours.iloc[0]
FORMATTED_HOURS = f'{STATEWIDE_HOURS:,}' 

display(
    HTML(
        f"<h3>{FORMATTED_HOURS} total service hours statewide</h3>"
    )
)

In [ ]:
service_cols_dict = {
    "category": "Category",
    "service_hours": "Service Hours",
    "pct_service_hours": "% Service Hours",
    "unique_route": "# Routes",
    "pct_unique_route": "% Routes",
    "service_hours_per_route": "Service Hours per Route",
}

summary_styled = portfolio_utils.style_table(
    summary, 
    rename_cols = service_cols_dict, 
    integer_cols = ["Service Hours", "# Routes"],
    one_decimal_cols = ["Service Hours per Route"],
    left_align_cols = "first",
    center_align_cols = "all",
    custom_format_cols = {'{:.1%}': ["% Service Hours", "% Routes"]},
    display_table = True
)

## Reliability (Delay)

Note: Not every route has GTFS Real-Time information, which supplies delay data.

In [ ]:
delay_df = df[df.merge_delay=="both"]
delay_summary = report_metrics.get_delay_summary_table(delay_df)

In [ ]:
STATEWIDE_DELAY = delay_summary.delay_hours.sum()
FORMATTED_HOURS = f'{STATEWIDE_DELAY:,g}' 

display(
    HTML(
        f"<h3>{FORMATTED_HOURS} total delay hours statewide</h3>"
    )
)

In [ ]:
delay_cols_dict = {
    "category": "Category",
    "delay_hours": "Total Delay Hours",
    "pct_delay_hours": "% Delay Hours",
    "unique_route": "# Routes",
    "pct_unique_route": "% Routes",
    "delay_hours_per_route": "Delay Hours per Route",
}

delay_summary_styled = portfolio_utils.style_table(
    delay_summary, 
    rename_cols = delay_cols_dict, 
    integer_cols = ["Total Delay Hours", "# Routes"],
    two_decimal_cols = ["Delay Hours per Route"],
    left_align_cols = "first",
    center_align_cols = "all",
    custom_format_cols = {'{:.1%}': ["% Delay Hours", "% Routes"]},
    display_table = True
)

## By District

In [ ]:
# Have some rows where district is missing, 
# but only for intersects_shn and other categories
# focus on just the on_shn category and do district breakdown
#df[(df.District.isna())].category.value_counts()

# Chart utils
WIDTH = 300
HEIGHT = 200

In [ ]:
by_district_service = report_metrics.by_district_on_shn_breakdown(
    df, ["service_hours", "unique_route"])

bar_total = (report_charts.make_bar(by_district_service, "service_hours")
             .properties(width=WIDTH, height=HEIGHT)  
            )
bar_avg = (report_charts.make_bar(by_district_service, "avg_service_hours")
           .properties(width=WIDTH, height=HEIGHT)
          )

service_hours_chart = report_charts.configure_hconcat_charts(
    [bar_avg, bar_total], 
    x_scale="independent", 
    y_scale="independent", 
    chart_title="Service Hours by District")

service_hours_chart

In [ ]:
by_district_delay = report_metrics.by_district_on_shn_breakdown(
    df, ["delay_hours", "unique_route"]
)

bar_total = (report_charts.make_bar(by_district_delay, "delay_hours")
             .properties(width=WIDTH, height=HEIGHT)      
            )
bar_avg = (report_charts.make_bar(by_district_delay, "avg_delay_hours")
           .properties(width=WIDTH, height=HEIGHT)
          )

delay_hours_chart = report_charts.configure_hconcat_charts(
    [bar_avg, bar_total], 
    x_scale="independent", 
    y_scale="independent", 
    chart_title="Delay Hours by District")

delay_hours_chart

## Map of Routes by Category

In [ ]:
def data_for_viz(df: gpd.GeoDataFrame):
    gdf = report_metrics.clean_up_category_values(df)
    
    # line must fall within CA
    ca = hq_catalog.ca_boundary.read().to_crs(f"EPSG: {gdf.crs.to_epsg()}")

    gdf = gpd.sjoin(
        gdf,
        ca,
        how = "inner",
        predicate = "within",
    ).drop(columns= ["index_right"])

    # Buffer to style the line, project to WGS84 for folium
    gdf = gdf.assign(
        geometry = (gdf.geometry.to_crs(geography_utils.CA_StatePlane)
                    .buffer(250).simplify(tolerance=100)
                    .to_crs(geography_utils.WGS84)
                   )
    )
    
    # Drop columns that shouldn't get displayed in tooltip
    drop_cols = ["_merge", "merge_delay", "State", "unique_route"]
    
    gdf2 = gdf.drop(columns = drop_cols)
    
    return gdf2


gdf = data_for_viz(df)

### All Routes (modes: rail, bus, ferry, unknown)

In [ ]:
def make_map(gdf: gpd.GeoDataFrame): 
    m = gdf.explore(
        "category", categorical=True,
       cmap = [cp.CALITP_CATEGORY_BRIGHT_COLORS[0], 
                cp.CALITP_CATEGORY_BRIGHT_COLORS[1],
                cp.CALITP_CATEGORY_BRIGHT_COLORS[2]],
        tiles = "Carto DB Positron"
    )

    display(m)

make_map(gdf)

### Rail / Ferry / Unknown Routes

In [ ]:
include = ["Rail", "Ferry", "Unknown"]
make_map(gdf[gdf.route_type_name.isin(include)])

### Bus Routes

In [ ]:
include = ["Bus"]
make_map(gdf[gdf.route_type_name.isin(include)])